In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
from collections import OrderedDict
from pprint import pprint

In [2]:
df_v_all = pd.DataFrame(columns=('Name','Link','API','Label','v_id', 'Total', 'Page_total', 'type_filter'))

In [3]:
#카테고리별 상황판 만들기 : 1차 카테고리 / 2차 카테고리 / 3차 카테고리 / 카테고리 URL / 페이지네이션 / 상품URL
req = urllib.request
url_v_all = "https://thrivemarket.com/sitemap"
req_v_all = req.urlopen(url_v_all).read()
bs_v_all = BeautifulSoup(req_v_all, "lxml")
bs_v_all = bs_v_all.find_all('ol')[1:5]
bs_v_all = [i.find_all('li')[1:] for i in bs_v_all]
bs_v_upzip = []
for v in bs_v_all:
    for n in v:
        bs_v_upzip.append(n)

In [4]:
bs_v_all_name = [ i.get_text() for i in bs_v_upzip]
bs_v_all_link = [ i.a.get('href') for i in bs_v_upzip]
df_v_all['Name'] = bs_v_all_name
df_v_all['Link'] = bs_v_all_link
df_v_all

,Name,Link,API,Label,v_id,Total,Page_total,type_filter
0,Gluten-Free,https://thrivemarket.com/lifestyle/gluten-free,NaN,NaN,NaN,NaN,NaN,NaN
1,Moms,https://thrivemarket.com/lifestyle/moms,NaN,NaN,NaN,NaN,NaN,NaN
2,Paleo,https://thrivemarket.com/lifestyle/paleo,NaN,NaN,NaN,NaN,NaN,NaN
3,Raw,https://thrivemarket.com/lifestyle/raw,NaN,NaN,NaN,NaN,NaN,NaN
4,Staples,https://thrivemarket.com/lifestyle/staples,NaN,NaN,NaN,NaN,NaN,NaN
5,Vegan,https://thrivemarket.com/lifestyle/vegan,NaN,NaN,NaN,NaN,NaN,NaN
6,Certified Gluten-Free,https://thrivemarket.com/certifications/certif...,NaN,NaN,NaN,NaN,NaN,NaN
7,Certified Kosher,https://thrivemarket.com/certifications/certif...,NaN,NaN,NaN,NaN,NaN,NaN
8,Certified Organic,https://thrivemarket.com/certifications/certif...,NaN,NaN,NaN,NaN,NaN,NaN
9,Certified Vegan,https://thrivemarket.com/certifications/certif...,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
Label = []
v_id = []
Total = []
type_filter = []
#라벨이랑 카테고리ID 갖고오기
for i in bs_v_all_link:
    link_for_api = i
    req_for_api = req.urlopen(link_for_api).read()
    bs_for_api = BeautifulSoup(req_for_api, "lxml")
    str_for_api = str(bs_for_api.find_all(string=re.compile('(filter":)(.*?)(,"productDetails")')))
    r = re.compile('"applied_filters":(.*?),"did_you_mean"')
    label_id = r.search(str_for_api).group()
    label = label_id.split('"')[11]
    _id = label_id.split('"')[7]
    _type = label_id.split('"')[3]
    Label.append(label)
    v_id.append(_id)
    type_filter.append(_type)
    #
    r2 = re.compile('"","total":(.*?),"errors"')
    total_tra = r2.search(str_for_api).group()[:-9]
    total = total_tra.split(':')
    Total.append(total[-1])
    print('ok', bs_v_all_link.index(i),i )

ok 0 https://thrivemarket.com/lifestyle/gluten-free
ok 1 https://thrivemarket.com/lifestyle/moms
ok 2 https://thrivemarket.com/lifestyle/paleo
ok 3 https://thrivemarket.com/lifestyle/raw
ok 4 https://thrivemarket.com/lifestyle/staples
ok 5 https://thrivemarket.com/lifestyle/vegan
ok 6 https://thrivemarket.com/certifications/certified-gluten-free
ok 7 https://thrivemarket.com/certifications/certified-kosher
ok 8 https://thrivemarket.com/certifications/certified-organic
ok 9 https://thrivemarket.com/certifications/certified-vegan
ok 10 https://thrivemarket.com/certifications/fair-trade-certified
ok 11 https://thrivemarket.com/certifications/non-gmo-project-verified
ok 12 https://thrivemarket.com/certifications/whole30-approved
ok 13 https://thrivemarket.com/ingredients/alcohol-free
ok 14 https://thrivemarket.com/ingredients/big-8-allergen-free
ok 15 https://thrivemarket.com/ingredients/caffeine-free
ok 16 https://thrivemarket.com/ingredients/cholesterol-free
ok 17 https://thrivemarket.co

In [7]:
Count_page = [ int(i)//120 + 1 for i in Total]
Count_page = [ str(k) for k in Count_page]
df_v_all['Page_total'] = Count_page
df_v_all['Label'] = Label
df_v_all['v_id'] = v_id
df_v_all['Total'] = Total
df_v_all['type_filter'] = type_filter

In [8]:
v_id = df_v_all['v_id'].tolist()
page_range =[int(i) for i in df_v_all['Page_total'].tolist()]

In [11]:
df_product = pd.DataFrame(columns=('category', 'v_id','manufacturer', 'title', 'short_description', 'review_score', 'review_count', 'msrp', 'price', 'url'))
all_url = []

# C_id와 page_range에 맞춰서 url 만들어주기
for i,v in enumerate(v_id):
    url_base = 'https://api.thrivemarket.com/products?'
    filter_type = df_v_all.loc[i].type_filter
    c_id = str(v) # 루프
    for k in range(1, page_range[i] + 1):
        p_num = str(k) # 루프
        api_url = url_base + 'filter[' + filter_type + ']=' + c_id + '&cur_page=' + p_num + '&page_size=120'
        all_url.append(api_url)
all_url

['https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=1&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=2&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=3&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=4&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=5&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=6&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=7&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=8&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=9&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=10&page_size=120',
 'https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=11&page_size=120',
 'https://api.thriv

In [12]:
#api url 넣으면 원하는 값들 뽑아주기
for u in all_url:
    req_api_url = requests.get(u)
    bs_api_url = BeautifulSoup(req_api_url.content, "lxml").find('p')
    text_api_url = bs_api_url.text
    print('TRY CHECK URL:', all_url.index(u), u)
    try:
        data = json.loads(text_api_url, object_pairs_hook=OrderedDict)
        print('OK URL:', all_url.index(u), u)
    except:
        print('  Err  !')
        pass
    # data_2 = json.loads(text_api_url)
    want_to_get = [ 'manufacturer', 'title', 'short_description', 'review_score', 'review_count', 'msrp', 'price', 'url']
    filter_type = list(data['applied_filters'])[0]
    print(data['applied_filters'][filter_type][0]['label'])
    for k in range(len(data['products'])):
        a = [data['applied_filters'][filter_type][0]['label']]
        b = [data['applied_filters'][filter_type][0]['key']]
        c = a + b + [ data['products'][k][i] for i in want_to_get]
        df_product.loc[len(df_product.index.tolist()) + 1] = c

TRY CHECK URL: 0 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=1&page_size=120
OK URL: 0 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=1&page_size=120
Gluten-Free
TRY CHECK URL: 1 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=2&page_size=120
OK URL: 1 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=2&page_size=120
Gluten-Free
TRY CHECK URL: 2 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=3&page_size=120
OK URL: 2 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=3&page_size=120
Gluten-Free
TRY CHECK URL: 3 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=4&page_size=120
OK URL: 3 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=4&page_size=120
Gluten-Free
TRY CHECK URL: 4 https://api.thrivemarket.com/products?filter[sub_brand]=938&cur_page=5&page_size=120
OK URL: 4 https://api.thrivemarket.com/products?filter[sub_bra

TRY CHECK URL: 40 https://api.thrivemarket.com/products?filter[sub_brand]=935&cur_page=6&page_size=120
OK URL: 40 https://api.thrivemarket.com/products?filter[sub_brand]=935&cur_page=6&page_size=120
Paleo
TRY CHECK URL: 41 https://api.thrivemarket.com/products?filter[sub_brand]=935&cur_page=7&page_size=120
OK URL: 41 https://api.thrivemarket.com/products?filter[sub_brand]=935&cur_page=7&page_size=120
Paleo
TRY CHECK URL: 42 https://api.thrivemarket.com/products?filter[sub_brand]=936&cur_page=1&page_size=120
OK URL: 42 https://api.thrivemarket.com/products?filter[sub_brand]=936&cur_page=1&page_size=120
Raw
TRY CHECK URL: 43 https://api.thrivemarket.com/products?filter[sub_brand]=936&cur_page=2&page_size=120
OK URL: 43 https://api.thrivemarket.com/products?filter[sub_brand]=936&cur_page=2&page_size=120
Raw
TRY CHECK URL: 44 https://api.thrivemarket.com/products?filter[sub_brand]=936&cur_page=3&page_size=120
OK URL: 44 https://api.thrivemarket.com/products?filter[sub_brand]=936&cur_page=3

TRY CHECK URL: 80 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=4&page_size=120
OK URL: 80 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=4&page_size=120
Certified Gluten-Free
TRY CHECK URL: 81 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=5&page_size=120
OK URL: 81 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=5&page_size=120
Certified Gluten-Free
TRY CHECK URL: 82 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=6&page_size=120
OK URL: 82 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=6&page_size=120
Certified Gluten-Free
TRY CHECK URL: 83 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=7&page_size=120
OK URL: 83 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=789&cur_page=7&page_size=120
Certified Gluten-Free
TRY CHECK UR

TRY CHECK URL: 114 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=5&page_size=120
OK URL: 114 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=5&page_size=120
Non-GMO Project Verified
TRY CHECK URL: 115 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=6&page_size=120
OK URL: 115 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=6&page_size=120
Non-GMO Project Verified
TRY CHECK URL: 116 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=7&page_size=120
OK URL: 116 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=7&page_size=120
Non-GMO Project Verified
TRY CHECK URL: 117 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=8&page_size=120
OK URL: 117 https://api.thrivemarket.com/products?filter[qualities_certs_awards]=791&cur_page=8&page_size=120
Non-GMO Project V

TRY CHECK URL: 147 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=2&page_size=120
OK URL: 147 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=2&page_size=120
Dairy Free
TRY CHECK URL: 148 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=3&page_size=120
OK URL: 148 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=3&page_size=120
Dairy Free
TRY CHECK URL: 149 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=4&page_size=120
OK URL: 149 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=4&page_size=120
Dairy Free
TRY CHECK URL: 150 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=5&page_size=120
OK URL: 150 https://api.thrivemarket.com/products?filter[qualities_health_ingredients]=895&cur_page=5&page_size=120
Dairy Free


...

TRY CHECK URL: 344 https://api.thrivemarket.com/products?filter[qualities_environment_social]=852&cur_page=2&page_size=120
OK URL: 344 https://api.thrivemarket.com/products?filter[qualities_environment_social]=852&cur_page=2&page_size=120
Certified B Corporation
TRY CHECK URL: 345 https://api.thrivemarket.com/products?filter[qualities_environment_social]=852&cur_page=3&page_size=120
OK URL: 345 https://api.thrivemarket.com/products?filter[qualities_environment_social]=852&cur_page=3&page_size=120
Certified B Corporation
TRY CHECK URL: 346 https://api.thrivemarket.com/products?filter[qualities_environment_social]=852&cur_page=4&page_size=120
OK URL: 346 https://api.thrivemarket.com/products?filter[qualities_environment_social]=852&cur_page=4&page_size=120
Certified B Corporation
TRY CHECK URL: 347 https://api.thrivemarket.com/products?filter[qualities_environment_social]=852&cur_page=5&page_size=120
OK URL: 347 https://api.thrivemarket.com/products?filter[qualities_environment_social]=8

TRY CHECK URL: 377 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur_page=3&page_size=120
OK URL: 377 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur_page=3&page_size=120
Family-Owned Business
TRY CHECK URL: 378 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur_page=4&page_size=120
OK URL: 378 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur_page=4&page_size=120
Family-Owned Business
TRY CHECK URL: 379 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur_page=5&page_size=120
OK URL: 379 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur_page=5&page_size=120
Family-Owned Business
TRY CHECK URL: 380 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur_page=6&page_size=120
OK URL: 380 https://api.thrivemarket.com/products?filter[qualities_environment_social]=848&cur

TRY CHECK URL: 410 https://api.thrivemarket.com/products?filter[qualities_environment_social]=841&cur_page=6&page_size=120
OK URL: 410 https://api.thrivemarket.com/products?filter[qualities_environment_social]=841&cur_page=6&page_size=120
Non-Toxic
TRY CHECK URL: 411 https://api.thrivemarket.com/products?filter[qualities_environment_social]=841&cur_page=7&page_size=120
OK URL: 411 https://api.thrivemarket.com/products?filter[qualities_environment_social]=841&cur_page=7&page_size=120
Non-Toxic
TRY CHECK URL: 412 https://api.thrivemarket.com/products?filter[qualities_environment_social]=841&cur_page=8&page_size=120
OK URL: 412 https://api.thrivemarket.com/products?filter[qualities_environment_social]=841&cur_page=8&page_size=120
Non-Toxic
TRY CHECK URL: 413 https://api.thrivemarket.com/products?filter[qualities_environment_social]=1431&cur_page=1&page_size=120
OK URL: 413 https://api.thrivemarket.com/products?filter[qualities_environment_social]=1431&cur_page=1&page_size=120
Pasture Rais

TRY CHECK URL: 443 https://api.thrivemarket.com/products?filter[qualities_environment_social]=1048&cur_page=1&page_size=120
OK URL: 443 https://api.thrivemarket.com/products?filter[qualities_environment_social]=1048&cur_page=1&page_size=120
Safe for septic tank
TRY CHECK URL: 444 https://api.thrivemarket.com/products?filter[qualities_environment_social]=851&cur_page=1&page_size=120
OK URL: 444 https://api.thrivemarket.com/products?filter[qualities_environment_social]=851&cur_page=1&page_size=120
Sourced Direct From Farmers
TRY CHECK URL: 445 https://api.thrivemarket.com/products?filter[qualities_environment_social]=851&cur_page=2&page_size=120
OK URL: 445 https://api.thrivemarket.com/products?filter[qualities_environment_social]=851&cur_page=2&page_size=120
Sourced Direct From Farmers
TRY CHECK URL: 446 https://api.thrivemarket.com/products?filter[qualities_environment_social]=851&cur_page=3&page_size=120
OK URL: 446 https://api.thrivemarket.com/products?filter[qualities_environment_so

TRY CHECK URL: 476 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=7&page_size=120
OK URL: 476 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=7&page_size=120
Sustainably farmed
TRY CHECK URL: 477 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=8&page_size=120
OK URL: 477 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=8&page_size=120
Sustainably farmed
TRY CHECK URL: 478 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=9&page_size=120
OK URL: 478 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=9&page_size=120
Sustainably farmed
TRY CHECK URL: 479 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=10&page_size=120
OK URL: 479 https://api.thrivemarket.com/products?filter[qualities_environment_social]=853&cur_page=10

In [13]:
writer = pd.ExcelWriter('[VALUE]_raw_list.xlsx')
df_v_all.to_excel(writer, 'raw_list')
writer.save()
writer = pd.ExcelWriter('[VALUE]_products_list.xlsx')
df_product.to_excel(writer, 'products_list')
writer.save()

In [ ]:
### 테스트 코드
link_for_api = 'https://thrivemarket.com/certifications/certified-organic'
req_for_api = req.urlopen(link_for_api).read()
bs_for_api = BeautifulSoup(req_for_api, "lxml")
str_for_api = str(bs_for_api.find_all(string=re.compile('(filter":)(.*?)(,"productDetails")')))
r = re.compile('"applied_filters":(.*?),"did_you_mean":')
label_id = r.search(str_for_api).group()
label = label_id.split('"')
r2 = re.compile('"","total":(.*?),"errors"')
total_tra = r2.search(str_for_api).group()[:-9]
total = total_tra.split(':')
label[3]